In [ ]:
# Imports
# Standard Libraries of Python
import sys
import time
import random
import itertools
from collections import Counter
from decimal import Decimal, ROUND_HALF_UP, getcontext
from typing import Union
getcontext().prec = 5

# Dependencies
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series

# Libraries made for this Proyect
from src.parse import draw_generator, Criteria
ruta_carpeta = 'data/simulation_result/'

In [ ]:
# Test of Tomorrow Numbers
euromillions = Criteria()

euromillions.groups_info()
euromillions.apply_transformation()
euromillions.count_skips()
euromillions.skips_for_last_12_draws()
euromillions.get_natural_rotations()
euromillions.numbers_clasification()

euromillions.year_criterion()
euromillions.rotation_criterion()
euromillions.position_criterion()
euromillions.group_criterion()
euromillions.numbers_of_tomorrow()

In [ ]:
euromillions.scrap

In [ ]:
# Classes to be defined
class Tickets():
    def __init__(self,euromillions: Criteria) -> object:
        # Inherits all the instances of Euro Millions
        self.euromillions = euromillions

        # Value of control
        self._s_n = 0

        # Counter for recommended numbers
        self._recommended_numbers_selected = 0

        # Counter for not recommended numbers
        self._not_recommended_numbers_selected = 0

        # List to store the selected numbers
        self._selected_numbers = []

    def draw_skips(self) -> DataFrame:
        # DataFrame to be populated
        self.d_skips = pd.DataFrame(columns=['nro1','nro2','nro3','nro4','nro5'])

        for index, row in euromillions.counts.iterrows():
            # Index to be updated. Row to be analize
            # Row to be populated
            new_row = []

            for column, value in row.items():
                # We search the value in each column of euromillions.counts, within the row of reference
                if value == 0:
                    try:
                        aus = euromillions.counts.loc[index - 1, column]
                    except (KeyError,IndexError):
                        aus = 0
                    if pd.isna(aus):
                        aus = 0
                    new_row.append(aus)
        
            while len(new_row) < 5:
                new_row.append(0)
        
            self.d_skips.loc[index] = new_row
        
        return self.d_skips
        
    def skips_evaluation(self) -> DataFrame:
        self.evaluation = pd.DataFrame(columns=['0','5','7','10','13'])
        counts = pd.DataFrame(0, index=self.d_skips.index, columns=self.evaluation.columns)

        counts['0'] = self.d_skips.apply(lambda row: row.eq(0).sum(),axis=1)
        counts['5'] = self.d_skips.apply(lambda row: row.between(0,5).sum(),axis=1)
        counts['7'] = self.d_skips.apply(lambda row: row.between(0,7).sum(),axis=1)
        counts['10'] = self.d_skips.apply(lambda row: row.between(0,10).sum(),axis=1)
        counts['13'] = self.d_skips.apply(lambda row: row.between(0,13).sum(),axis=1)

        self.evaluation = pd.concat([self.evaluation,counts],ignore_index=True)
        self.evaluation = self.evaluation.set_index(pd.RangeIndex(1, len(self.evaluation) + 1))

        return self.evaluation

    def __df_numbers(self) -> Series:
        self._df_values = euromillions.last_draw.groupby('skips')['number'].apply(lambda x: list(x)).reset_index().set_index('skips')
        self._df_values = self._df_values.rename_axis('number').rename_axis(None)
        self._df_values['number'] = self._df_values['number'].apply(lambda x: sorted(x))
        return self._df_values['number']

    def __list_of_numbers(self, idx: int, n_category: DataFrame) -> list | DataFrame:
        self.numbers = self.__df_numbers()

        assert self.numbers is not None and not self.numbers.empty, "There are no available numbers to select. Please check the Series from the last draw."

        row = self.numbers.loc[0] if idx == 0 else self.numbers.loc[idx]
        available_numbers = n_category[n_category['numbers'].isin(row)]['numbers'].tolist()

        if idx == 0 and self._s_n > 0:
            raise ValueError("Idx 0 was already selected. Please, put another index.")
        elif available_numbers:
            return available_numbers, n_category
        elif not available_numbers:
            not_n_category = euromillions.not_recommended_numbers
            available_numbers = not_n_category[not_n_category['numbers'].isin(row)]['numbers'].tolist()
            return available_numbers, not_n_category
        else:
            raise ValueError("The list of numbers to be selected is empty.")
    
    def __select_number(self, idx: int, n_category: DataFrame) -> int:
        numbers, n_category = self.__list_of_numbers(idx,n_category)
        self.__operation(numbers,idx,n_category)

    def __operation(self, available_numbers: list, idx: int, n_category: DataFrame) -> int:
        match idx:
            case 0:
                rng = 1
            case _:
                if self._s_n > 0 and self._s_n <= 2:
                    rng = random.randint(1,2)
                else:
                    rng = 0

        if rng == 2 and len(available_numbers) > 2:
            selected_numbers = np.random.choice(available_numbers, size=rng, replace=False)
            self._selected_numbers.extend(selected_numbers)
            self._s_n += rng
            self.__remove_number(selected_numbers,n_category)
            self.__sum_selected_number(selected_numbers)
        elif rng == 1 or (rng == 0 and len(available_numbers) > 0):
            selected_number = np.random.choice(available_numbers, replace=False)
            self._selected_numbers.append(selected_number)
            self._s_n += 1
            self.__remove_number(selected_number,n_category)
            self.__sum_selected_number(selected_number)
    
    def __remove_number(self, value:Union[np.int32,list], n_category: DataFrame) -> DataFrame:
        n_category = n_category.drop(n_category.loc[n_category['numbers'] == value].index).reset_index(drop=True)
        n_category = self.__probability(n_category)
        return n_category
    
    def __probability(self,n_category:DataFrame) -> DataFrame:
        probability = 1 / len(n_category)
        n_category['criteria'] = n_category['criteria'] * (1 + probability)
        return n_category

    def __sum_selected_number(self,selected_number:int) -> int:
        if euromillions.recommended_numbers[euromillions.recommended_numbers['numbers'].isin(selected_number)]['numbers'].tolist():
            self._recommended_numbers_selected += 1
        else:
            self._not_recommended_numbers_selected += 1
    
    def first_number(self) -> int:
        self.__select_number(0,euromillions.recommended_numbers)
    
    def suggested_numbers(self) -> list:
        skips = self._df_values['skips'].unique().tolist()
        while self._recommended_numbers_selected < 6:
            idx = np.random.choice(skips,size=1,replase=False)
            try:
                self.__select_number(idx, euromillions.recommended_numbers)
            except ValueError:
                while idx == 0:
                    idx = np.random.choice(skips,size=1,replase=False)

In [ ]:
lotto = Tickets(euromillions)

In [ ]:
lotto.draw_skips()

In [ ]:
lotto.skips_evaluation()

In [ ]:
lotto.first_number()

In [ ]:
lotto._selected_numbers

In [ ]:
# Test of lottery tickets (dev)
lotto = Tickets(euromillions)
lotto.draw_skips()

In [ ]:
# Test of Euromillions Analysis
start_time = time.time()

success = []
failure = []
results = []
euromillions = Criteria()
size = len(euromillions.scrap)

for draw in draw_generator(size):
    db_slice = euromillions.scrap.head(draw)
    euromillions.db = db_slice
    euromillions.groups_info()
    euromillions.apply_transformation()
    euromillions.count_skips()
    euromillions.skips_for_last_12_draws()
    euromillions.get_natural_rotations()
    euromillions.numbers_clasification()

    euromillions.year_criterion()
    euromillions.rotation_criterion()
    euromillions.position_criterion()
    euromillions.group_criterion()
    euromillions.numbers_of_tomorrow()

    row = euromillions.scrap.loc[draw,['nro1','nro2','nro3','nro4','nro5']]
    column = euromillions.recommended_numbers.loc[:,'numbers']
    if len(column) < 25:
        missing_rows = 25 - len(column)
        first_not_recommended_numbers = euromillions.not_recommended_numbers.head(missing_rows)
        column = pd.concat([euromillions.recommended_numbers['numbers'], first_not_recommended_numbers.iloc[:, 0]]).reset_index(drop=True)
    else:
        result = np.isin(column,row).sum()
        success.append(result)
        failure.append(5 - result)

    # recommended_numbers = euromillions.recommended_numbers['numbers'].sample(6).tolist()
    # not_recommended_numbers = euromillions.not_recommended_numbers['numbers'].sample(4).tolist()
    # total_numbers = recommended_numbers + not_recommended_numbers
    # combinations = list(itertools.combinations(total_numbers, 5))

    # draw_results = []

    # for combination in combinations:
    #     counts = np.isin(row, combination).sum()
    #     draw_results.append(counts)

    # results.append(draw_results)
    # file_name = 'results.csv'
    # df_resultado = pd.DataFrame({'Combination': combinations, 'Count': draw_results})

    # Obtener el número máximo de conteo
    # max_count = max(draw_results)

    # if max_count == 0:
    #     print('FUCK!')
    # elif max_count == 1:
    #     print('Too bad')
    # elif max_count == 2:
    #     print('meh')
    # elif max_count == 3:
    #     print('now we are talking')
    # elif max_count == 4:
    #     print('holy shit!')
    # elif max_count == 5:
    #     print('HELL YEAH')

    sys.stdout.write(f"\ri = {draw}")
    sys.stdout.flush()
    
c = Counter(success)
x = Counter(failure)

# Print quantity of hits per draw, with recommended numbers, and random numbers aside
for i in range(0,6):
    hits = (success.count(i)/len(success))*100
    print(f"{i} hits: {c[i]}\n{round(hits,2)}%")

In [ ]:
euromillions.scrap

In [ ]:
random_succes = []
random_failure = []
for draw in draw_generator(size):
    db_resultados = db_slice.head(draw)
    row = euromillions.scrap.loc[draw,['nro1','nro2','nro3','nro4','nro5']]
    random_numbers = np.random.choice(range(1,51),size=25,replace=False)
    result = np.isin(row, random_numbers).sum()
    random_succes.append(result)
    random_failure.append(5-result)
    sys.stdout.write(f"\ri = {draw}")
    sys.stdout.flush()

y = Counter(random_succes)
z = Counter(random_failure)

# Print quantity of hits per draw, with recommended numbers, and random numbers aside
for e in range(0,6):
    nohits = (random_succes.count(e)/len(random_succes))*100
    print(f"{e} random hits: {y[e]}\n{round(nohits,2)}%")